# Forecasting Consensus Expectations: Nonfarm Payrolls (NFP) 

## Data preprocessing

**Imports**

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

from tqdm.auto import tqdm
from scipy import stats, special
from scipy.optimize import brentq
from collections import defaultdict
from itertools import product
from scipy.stats import t as student_t, norm, binomtest, jarque_bera

**Load & Preprocess**

In [3]:
HIST_PATH = "../raw/historical.xls"
JUNE_PATH = "../raw/june_forecasts.xls"

hist_raw = pd.read_html(HIST_PATH)[0]
june_raw = pd.read_html(JUNE_PATH)[0]

In [4]:
hist_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,712,713,714,715,716,717,718,719,720,721
0,NFP TCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summary,NaN,Actual,Economist,NaN,Peter Dixon,NaN,Sonia Meskin,NaN,Nikola Mirtchev,...,NaN,James A Shugg,NaN,John McAuley,NaN,Nash Peyton,NaN,Stephanie Roth,NaN,Crandall/Jordan
2,Release Date,Observation Date,NaN,Firm,As of,NaN,As of,NaN,As of,3d Currency Management Limited,...,As of,Westpac Banking Corp,As of,Wilkinson Boyd Captl Mrkts Inc,As of,Wilmington Trust Company,As of,Wolfe Research LLC,As of,Wrightson ICAP LLC
3,02/04/2000,01/31/2000,387,NaN,--,--,--,--,--,--,...,--,--,02/04/2000,200,--,--,--,--,02/04/2000,225
4,03/03/2000,02/29/2000,43,NaN,--,--,--,--,--,--,...,--,--,02/29/2000,200,--,--,--,--,03/03/2000,180


In [5]:
june_raw.head()

,0,1,2,3,4,5
0,NFP TCH Index,NaN,NaN,NaN,NaN,NaN
1,NaN,Release Date,6/6/2025,NaN,NaN,NaN
2,NaN,Time,08:30,NaN,NaN,NaN
3,NaN,Country/Region,US,NaN,NaN,NaN
4,NaN,Event,Change in Nonfarm Payrolls,NaN,NaN,NaN


In [6]:
# Raw data is messy. Want to process this into a long table format for our smart NFP forecasting (where each row is a forecast by a single economist/sell-side institution)

name_row = hist_raw.iloc[1]
firm_row = hist_raw.iloc[2]

In [7]:
base = hist_raw.iloc[3:, :3].copy()
base.columns = ["release_date", "period", "actual"]
base["release_date"] = pd.to_datetime(base["release_date"])
base["period"] = pd.to_datetime(base["period"])
base["actual"] = pd.to_numeric(base["actual"], errors="coerce")

In [8]:
# numeric forecast cols
econ_cols = [idx for idx, val in name_row.items()
             if isinstance(val,str) and val not in {'Summary', 'Actual', 'Economist'}
]

In [9]:
# collect long dataframes, one per economist 

long_frames = []

for col in econ_cols:
    asof_col = col - 1
    
    tmp = base.copy()
    
    tmp["economist"] = name_row[col]
    tmp["firm"]      = firm_row[col]
    
    tmp["forecast"]  = pd.to_numeric(hist_raw.iloc[3:, col], errors="coerce")
    
    tmp["asof"]      = pd.to_datetime(hist_raw.iloc[3:, asof_col], errors="coerce")
    
    long_frames.append(tmp)
    
hist_long = pd.concat(long_frames, ignore_index=True)

C:\Users\nwong\AppData\Local\Temp\ipykernel_26280\311849067.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tmp["asof"]      = pd.to_datetime(hist_raw.iloc[3:, asof_col], errors="coerce")
C:\Users\nwong\AppData\Local\Temp\ipykernel_26280\311849067.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tmp["asof"]      = pd.to_datetime(hist_raw.iloc[3:, asof_col], errors="coerce")
C:\Users\nwong\AppData\Local\Temp\ipykernel_26280\311849067.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tmp["asof"]      = pd.to_datetime(hist_raw.iloc[3:, asof_col], errors="coerce")
C

In [10]:
hist_long

,release_date,period,actual,economist,firm,forecast,asof
0,2000-02-04,2000-01-31,387,Peter Dixon,NaN,NaN,NaT
1,2000-03-03,2000-02-29,43,Peter Dixon,NaN,NaN,NaT
2,2000-04-07,2000-03-31,416,Peter Dixon,NaN,NaN,NaT
3,2000-05-05,2000-04-30,340,Peter Dixon,NaN,NaN,NaT
4,2000-06-02,2000-05-31,231,Peter Dixon,NaN,340.0,2000-05-30
...,...,...,...,...,...,...,...
96667,2025-01-10,2024-12-31,256,Crandall/Jordan,Wrightson ICAP LLC,150.0,2024-12-20
96668,2025-02-07,2025-01-31,143,Crandall/Jordan,Wrightson ICAP LLC,125.0,2025-01-17
96669,2025-03-07,2025-02-28,151,Crandall/Jordan,Wrightson ICAP LLC,100.0,2025-02-14
96670,2025-04-04,2025-03-31,228,Crandall/Jordan,Wrightson ICAP LLC,80.0,2025-03-14


In [11]:
##Tidy June 2025 release file 

# Build mask to keep valid forecaster rows 
forecast_col = pd.to_numeric(june_raw[3], errors="coerce")

mask = june_raw[0].isna() & forecast_col.notna()

june_core = (
    june_raw.loc[mask, [1,2,3,4,5]]
    .rename(columns = {1: "economist",
                       2: "firm",
                       3: "forecast", 
                       4: "asof", 
                       5: "rank"})
)

june_core["forecast"] = forecast_col[mask]
june_core["asof"] = pd.to_datetime(june_core["asof"],
                                   errors="coerce")

# substitute firm name if economist is missing 
june_core["economist"] = june_core.apply(
    lambda r: r["economist"] if pd.notna(r["economist"]) else f"({r['firm']})",
    axis = 1
)

# Extract NFP print 
actual_mask = june_raw.apply(
    lambda r: r.astype(str).str.contains("Actual", case=False).any(), axis = 1
)
actual_val = (
    pd.to_numeric(june_raw.loc[actual_mask].stack(), errors = "coerce")
                  .dropna()
                  .iloc[0]
)

# hardcode release and reference month 
release_date = pd.to_datetime("2025-06-06") # BLS release day 
period_date = pd.to_datetime("2025-05-01")  # payroll month 

# match hist_long schema for merge

june_long = (
    june_core.assign(
        release_date = release_date,
        period = period_date,
        actual = actual_val
    )
    .loc[:, ["release_date", "period", "economist", "firm", 
             "forecast", "actual", "asof"]]
)

In [12]:
# verify that june matches the historical panel for stacking
june_long

,release_date,period,economist,firm,forecast,actual,asof
22,2025-06-06,2025-05-01,Stan Shipley,Evercore ISI,115.0,139.0,2025-06-05
23,2025-06-06,2025-05-01,Crandall/Jordan,Wrightson ICAP LLC,150.0,139.0,2025-05-16
24,2025-06-06,2025-05-01,Peter G Morici,University of Maryland,120.0,139.0,2025-05-21
25,2025-06-06,2025-05-01,Hugh A Johnson,Hugh Johnson Economics LLC,130.0,139.0,2025-05-23
26,2025-06-06,2025-05-01,Michael R Englund,Action Economics LLC,135.0,139.0,2025-05-23
...,...,...,...,...,...,...,...
96,2025-06-06,2025-05-01,(Vinland Capital Management II Gestora de Recu...,Vinland Capital Management II Gestora de Recur...,135.0,139.0,2025-06-05
97,2025-06-06,2025-05-01,Yonie Fanning,Mizrahi Tefahot Bank Limited,98.0,139.0,2025-06-05
98,2025-06-06,2025-05-01,Nicholas Van Ness,Credit Agricole CIB,115.0,139.0,2025-06-05
99,2025-06-06,2025-05-01,(Japan Digital Design Inc),Japan Digital Design Inc,155.0,139.0,2025-06-05


In [13]:
# Stack dfs (June + historical long formats)

# Stack panels  
nfp_long = pd.concat([hist_long, june_long], ignore_index = True, sort = False)


# Compute forecast error 
nfp_long["error"] = nfp_long["forecast"] - nfp_long["actual"]

# Per release, we only want the last forecast 
econ_forecasts_long = (
    nfp_long
    .sort_values(["release_date", "economist", "asof"])
    .groupby(["release_date", "economist"], as_index=False)
    .tail(1)
    .reset_index(drop=True)
)

In [14]:
df_full = econ_forecasts_long
df_full.head()

,release_date,period,actual,economist,firm,forecast,asof,error
0,2000-02-04,2000-01-31,387.0,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN
1,2000-02-04,2000-01-31,387.0,Alessandro Truppia,Aletti Gestielle Sgr Spa,NaN,NaT,NaN
2,2000-02-04,2000-01-31,387.0,Alexandre De Azara,Banco UBS SA,NaN,NaT,NaN
3,2000-02-04,2000-01-31,387.0,Alison Lynn Reaser,Point Loma Nazarene University,NaN,NaT,NaN
4,2000-02-04,2000-01-31,387.0,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN


In [15]:
# Quick sanity checks

# should only have one forecast per economist, per release
assert df_full.duplicated(subset=["release_date", "economist"]).sum() == 0,\
    "Duplicate (release date, economist) combinations detected"

# forecast timestamp has to be before release date. caveat: NA values will make the assertion fail, so we account NAs.
valid_asof = df_full["asof"].notna()
assert (df_full.loc[valid_asof, "asof"] <= df_full.loc[valid_asof, "release_date"]).all(),\
    "Some forecasts have asof (non-NA) after release date."

# each release date should have only one realized (actual) value 
assert (df_full.groupby("release_date")["actual"].nunique() == 1).all(), \
    "Multiple actual values found for the same release date"
    

# Note (economist, release_date) is not unique as some banks have multiple economist forecasts for a single release

In [16]:
# get median

median_by_release = (
    df_full.groupby("release_date")["forecast"]
    .median()
    .rename("median_forecast")
)

df_full = df_full.merge(median_by_release, on="release_date", how="left")

In [17]:
df_full.head()

,release_date,period,actual,economist,firm,forecast,asof,error,median_forecast
0,2000-02-04,2000-01-31,387.0,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN,267.5
1,2000-02-04,2000-01-31,387.0,Alessandro Truppia,Aletti Gestielle Sgr Spa,NaN,NaT,NaN,267.5
2,2000-02-04,2000-01-31,387.0,Alexandre De Azara,Banco UBS SA,NaN,NaT,NaN,267.5
3,2000-02-04,2000-01-31,387.0,Alison Lynn Reaser,Point Loma Nazarene University,NaN,NaT,NaN,267.5
4,2000-02-04,2000-01-31,387.0,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN,267.5


In [18]:
# Sanity output 
print("NAs: ")
print(df_full.isna().sum(), "\n")
print("Rows: ", econ_forecasts_long.shape[0])
print("Economists: ", econ_forecasts_long['economist'].nunique())
print("NFP releases: ", econ_forecasts_long['release_date'].nunique())

NAs: 
release_date           0
period                 0
actual                 0
economist              0
firm                 608
forecast           75507
asof               75507
error              75507
median_forecast        0
dtype: int64 

Rows:  96751
Economists:  340
NFP releases:  305


**Filter COVID, define directional label**

In [19]:
# Surprise = actual minus consensus (median forecast) for directional forecasting
df_full["surprise"] = df_full["actual"] - df_full["median_forecast"]

# Filter out COVID dates --> avoid penalizing naive model for erratic performance in pandemic 
df = df_full[~df_full["release_date"].between("2020-01-01", "2022-12-31")]
df.head()

,release_date,period,actual,economist,firm,forecast,asof,error,median_forecast,surprise
0,2000-02-04,2000-01-31,387.0,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN,267.5,119.5
1,2000-02-04,2000-01-31,387.0,Alessandro Truppia,Aletti Gestielle Sgr Spa,NaN,NaT,NaN,267.5,119.5
2,2000-02-04,2000-01-31,387.0,Alexandre De Azara,Banco UBS SA,NaN,NaT,NaN,267.5,119.5
3,2000-02-04,2000-01-31,387.0,Alison Lynn Reaser,Point Loma Nazarene University,NaN,NaT,NaN,267.5,119.5
4,2000-02-04,2000-01-31,387.0,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN,267.5,119.5


In [ ]:
OUT_DIR = "../out"          # 1-level up (Verition/) then into out/
DF_FILE       = "df.parquet"
DF_FULL_FILE  = "df_full.parquet"

# ensure directory exists 
os.makedirs(OUT_DIR, exist_ok=True)

df.to_parquet(os.path.join(OUT_DIR, DF_FILE),  engine="pyarrow", index=False)
df_full.to_parquet(os.path.join(OUT_DIR, DF_FULL_FILE), engine="pyarrow", index=False)

print(f"✔️  Saved df  ➜  {OUT_DIR}/{DF_FILE}")
print(f"✔️  Saved df_full ➜  {OUT_DIR}/{DF_FULL_FILE}")

✔️  Saved df  ➜  ../out/df.parquet
✔️  Saved df_full ➜  ../out/df_full.parquet
